In [1]:
import os

In [2]:
%pwd

'/home/fachruzaini/lazada-id-reviews/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
%pwd

'/home/fachruzaini/lazada-id-reviews'

### Data Preprocessing Config

This code will be apply in `src/LadazaIDReview/entity/config_entity.py`

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataDumpConfig:
    root_dir: Path
    reviews_path: Path
    input_train_path: Path
    input_test_path: Path
    output_train_path: Path
    output_test_path: Path
    params_test_size: float

@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    input_train_path: Path
    input_test_path: Path
    vectorized_train_path: Path
    vectorized_test_path: Path
    model_dir: Path
    vectorizer_model_path: Path

### Data Preprocessing Config Manager

This code will be apply in `src/LazadaIDReview/config/configurations.py`.

What we would do?
+ Drop null values
+ Splitting the dataset to train and test data
+ Vectorize text using `TFIDF`

As stated before; let’s load, select columns, and drop null values from dataset.

In [6]:
from LazadaIDReviews.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from LazadaIDReviews.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_dump_data_config(self) -> DataDumpConfig:
        """read data dump config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: PreprocessingConfig type
        """
        data_ingest_config = self.config.ingest_from_sql
        data_dump_config = self.config.dump_data
        dataset_params = self.params

        create_directories([data_dump_config.root_dir])

        config = DataDumpConfig(
            root_dir=data_dump_config.root_dir,
            reviews_path=data_ingest_config.reviews_path,
            input_train_path=data_dump_config.input_train_path,
            input_test_path=data_dump_config.input_test_path,
            output_train_path=data_dump_config.output_train_path,
            output_test_path=data_dump_config.output_test_path,
            params_test_size=dataset_params.TEST_SIZE
        )

        return config
    
    def get_preprocessing_data_config(self) -> DataPreprocessingConfig:
        """read preprocessing config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: PreprocessingConfig type
        """
        data_dump_config = self.config.dump_data
        vectorize_config = self.config.vectorize_data
        train_config = self.config.train_model

        create_directories([vectorize_config.root_dir])

        config = DataPreprocessingConfig(
            root_dir=vectorize_config.root_dir,
            input_train_path=Path(data_dump_config.input_train_path),
            input_test_path=Path(data_dump_config.input_test_path),
            vectorized_train_path=Path(vectorize_config.vectorized_train_path),
            vectorized_test_path=Path(vectorize_config.vectorized_test_path),
            model_dir=train_config.root_dir,
            vectorizer_model_path=Path(vectorize_config.vectorizer_model_path)
        )

        return config

### Perform Preprocessing

This code in `src/LazadaIDReview/components/preprocessing.py`.

In [ ]:
import joblib
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from LazadaIDReviews import logger

class DumpData:
    def __init__(self, config: DataDumpConfig):
        self.config = config

    def dump_data(self) -> None:
        """dump the splited dataset to data training and testing
        """
        logger.info(f"Read reviews file.")
        dataset_reviews = pd.read_csv(self.config.reviews_path)
        dataset = dataset_reviews[["rating", "reviewContent"]].copy()
        dataset.dropna(inplace=True)
        
        logger.info(f"Split reviews file to data train and test.")
        X_train, X_test, y_train, y_test = train_test_split(
            dataset["reviewContent"], 
            dataset["rating"], 
            test_size=self.config.params_test_size
        )
        
        logger.info(f"Dump data train into {self.config.input_train_path} directory.")
        X_train.to_pickle(self.config.input_train_path)
        X_test.to_pickle(self.config.input_test_path)
        
        logger.info(f"Dump data test into {self.config.input_test_path} directory.")
        y_train.to_pickle(self.config.output_train_path)
        y_test.to_pickle(self.config.output_test_path)

class Preprocessing:
    def __init__(self, config: DataPreprocessingConfig):
        self.config = config

    def vectorize_data(self) -> None:
        """vectorize the splited dataset and dump vectorizer model
        """
        vectorizer = TfidfVectorizer()
        
        logger.info(f"Load data train in {self.config.input_train_path}.")
        X_train = joblib.load(self.config.input_train_path)
        
        logger.info(f"Load data test in {self.config.input_test_path}.")
        X_test = joblib.load(self.config.input_test_path)
        
        logger.info(f"Vectorize the data.")
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)
        
        logger.info(f"Dump the vectorized data.")
        joblib.dump(X_train_vec, self.config.vectorized_train_path)
        joblib.dump(X_test_vec, self.config.vectorized_test_path)
        
        logger.info(f"Creating {self.config.model_dir} directory.")
        model_dir = str(self.config.model_dir)
        os.makedirs(model_dir, exist_ok=True)
        
        logger.info(f"Save the vectorizer model.")
        joblib.dump(vectorizer, self.config.vectorizer_model_path)

### Dump the Data Train and Data Test

This code in `src/LazadaIDReview/pipeline/step_02_preprocessing.py`.

In [9]:
try:
    config = ConfigurationManager()
    dump_data_config = config.get_dump_data_config()
    data_ingestion = DumpData(config=dump_data_config)
    data_ingestion.dump_data()
except Exception as e:
    logger.error(e)
    raise e

[2025-07-02 14:35:24,449: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-02 14:35:24,455: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2025-07-02 14:35:24,457: INFO: common: created directory at: artifacts]
[2025-07-02 14:35:24,459: INFO: common: created directory at: artifacts/data]
[2025-07-02 14:35:24,460: INFO: 821302001: Read reviews file.]
[2025-07-02 14:35:25,657: INFO: 821302001: Split reviews file to data train and test.]
[2025-07-02 14:35:25,673: INFO: 821302001: Dump data train into artifacts/data/X_train.pkl directory.]
[2025-07-02 14:35:25,779: INFO: 821302001: Dump data test into artifacts/data/X_test.pkl directory.]


**Debug**: Read data

In [10]:
X_train = joblib.load(dump_data_config.input_train_path)
X_train

41910     murah namun berkualitas. sound juga ok. sayang...
59760                         Packing rapih , produk sesuai
64575                                                mantab
11503                bagus, sesuai kebutuhan kerja, praktis
43546                               barangnya blm di terima
                                ...                        
179467    barang sudah sampai dengan rapi, dan cepat,  t...
17460     terima kasih barangnya udah sampai dan udah di...
119535    Tolong balas gan, harganya seriusan tuh 48" 1,...
94170                         barangnya bagus n mantap..👍👍👍
128584    cepat sekali datangnya, fd belum dicoba sih, s...
Name: reviewContent, Length: 21405, dtype: object

In [11]:
X_train.isnull().sum()

0

In [12]:
y_train = joblib.load(dump_data_config.output_train_path)
y_train

41910     5
59760     5
64575     5
11503     5
43546     1
         ..
179467    3
17460     4
119535    5
94170     5
128584    5
Name: rating, Length: 21405, dtype: int64

In [13]:
X_test = joblib.load(dump_data_config.input_test_path)
X_test

191809    sdh 2 kali order TV coocaa di lazada, pertama ...
203751           Barang sudah sampai ditempat,udah di pakai
202508    Recomended bgt, barang cepat sampai dan sesuai...
125832                   barang sesuai dengan pesanan trims
26944                                 ori, pengiriman cepat
                                ...                        
16978     Barang bagus sesuai deskripsi, harga jg terjan...
32392     Sangat amat kecewa hardisk tidak bisa terbaca ...
719             pengririman cepat dan barang sesuai pesanan
22690                   Paket dalam kondisi baik dan lancar
4577                            spek maksimal harga minimal
Name: reviewContent, Length: 85624, dtype: object

In [14]:
X_test.isnull().sum()

0

In [15]:
y_test = joblib.load(dump_data_config.output_test_path)
y_test

191809    5
203751    5
202508    5
125832    5
26944     5
         ..
16978     5
32392     2
719       5
22690     5
4577      5
Name: rating, Length: 85624, dtype: int64

### Vectorize the Data Train and Data Test

This code in `src/LazadaIDReview/pipeline/step_02_preprocessing.py`.

In [16]:
try:
    config = ConfigurationManager()
    preprocessing_config = config.get_preprocessing_data_config()
    preprocessing = Preprocessing(config=preprocessing_config)
    preprocessing.vectorize_data()
except Exception as e:
    logger.error(e)
    raise e

[2025-07-02 14:35:26,206: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-02 14:35:26,209: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2025-07-02 14:35:26,211: INFO: common: created directory at: artifacts]
[2025-07-02 14:35:26,213: INFO: common: created directory at: artifacts/preprocessing]
[2025-07-02 14:35:26,215: INFO: 821302001: Load data train in artifacts/data/X_train.pkl.]
[2025-07-02 14:35:26,252: INFO: 821302001: Load data test in artifacts/data/X_test.pkl.]
[2025-07-02 14:35:26,417: INFO: 821302001: Vectorize the data.]
[2025-07-02 14:35:27,783: INFO: 821302001: Dump the vectorized data.]
[2025-07-02 14:35:27,862: INFO: 821302001: Creating artifacts/models directory.]
[2025-07-02 14:35:27,863: INFO: 821302001: Save the vectorizer model.]


**Debug**: Read data

In [17]:
X_train_vec = joblib.load(preprocessing_config.vectorized_train_path)
X_train_vec

<21405x13716 sparse matrix of type '<class 'numpy.float64'>'
	with 251565 stored elements in Compressed Sparse Row format>

In [18]:
X_test_vec = joblib.load(preprocessing_config.vectorized_test_path)
X_test_vec

<85624x13716 sparse matrix of type '<class 'numpy.float64'>'
	with 984159 stored elements in Compressed Sparse Row format>